# PART- 2 `( Handling With Missing Values )`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
import seaborn as sns

import warnings

# Statistics functions
from scipy.stats import norm
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

/Users/saint/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Suppressing a warning 
warnings.filterwarnings("ignore") 

# It is a magic function that renders the figure in the notebook
%matplotlib inline 

# Changing the figure size of a seaborn axes 
sns.set(rc={"figure.figsize": (10, 6)})

# The style parameters control properties like the color of the background and whether a grid is enabled by default.
sns.set_style("whitegrid")

# To display maximum columns
pd.set_option('display.max_columns', None)

# To display maximum columns
# pd.set_option('display.max_rows', None)

In [3]:
import re 

def get_diff_category_column(Series:pd.Series, exclude=''',/\n''', pattern=r'''[,\n]| /''', strip='''\n' "!?|.,*+-_/][\\n''', diff_value=list()):  
    diff_value.clear()
    column = Series.dropna().apply(str).str.strip(strip)
    for row in column:
        if not any(x in exclude for x in row) and row not in diff_value:
            diff_value.append(row)
        else:
            for data in map(lambda x: x.strip(strip), filter(None, re.split(pattern, row))):
                if data not in diff_value:
                    diff_value.append(data)
    return dict(enumerate(sorted(diff_value)))

In [4]:
df = pd.read_csv("cleaned_autoscout.csv")
df.head()

,short_description,body_type,price_€,vat,km,registration,prev_owner,hp_kW,type,previous_owners,warranty,make,model,body_color,paint_type,body_color_original,upholstery,nr_of_doors,nr_of_seats,gearing_type,engine_displacement_cc,cylinders,weight_kg,drive_chain,fuel,co2e_g,emission_class,comfort&convenience,entertainment&media,extras,safety&security,gears,country_version,consumption_comb,consumption_city,consumption_country
0,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770.0,VAT deductible,56013.0,2016.0,2.0,66.0,Used,2.0,NaN,Audi,A1,Black,Metallic,Mythosschwarz,Cloth,5.0,5.0,Automatic,1422.0,3.0,1220.0,front,Diesel,99.0,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...",NaN,NaN,3.8,4.3,3.5
1,1.8 TFSI sport,Sedans,14500.0,Price negotiable,80000.0,2017.0,NaN,141.0,Used,NaN,NaN,Audi,A1,Red,NaN,NaN,Cloth,3.0,4.0,Automatic,1798.0,4.0,1255.0,front,Benzine,129.0,Euro 6,"Air conditioning,Automatic climate control,Hil...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Sport seats,Sport suspension,Voic...","ABS,Central door lock,Central door lock with r...",7.0,NaN,5.6,7.1,4.7
2,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640.0,VAT deductible,83450.0,2016.0,1.0,85.0,Used,1.0,NaN,Audi,A1,Black,Metallic,mythosschwarz metallic,Cloth,4.0,4.0,Automatic,1598.0,NaN,NaN,front,Diesel,99.0,Euro 6,"Air conditioning,Cruise control,Electrical sid...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Daytime running lights,D...",NaN,NaN,3.8,4.4,3.4
3,1.4 TDi Design S tronic,Sedans,14500.0,NaN,73000.0,2016.0,1.0,66.0,Used,1.0,NaN,Audi,A1,Brown,Metallic,NaN,NaN,3.0,4.0,Automatic,1422.0,3.0,1195.0,NaN,Diesel,99.0,Euro 6,"Air suspension,Armrest,Auxiliary heating,Elect...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport seats,Voice Control","ABS,Alarm system,Central door lock with remote...",6.0,NaN,3.8,4.3,3.5
4,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790.0,NaN,16200.0,2016.0,1.0,66.0,Used,1.0,NaN,Audi,A1,Black,Metallic,Mythosschwarz Metallic,Cloth,5.0,5.0,Automatic,1422.0,3.0,NaN,front,Diesel,109.0,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport package,Sport suspension,Vo...","ABS,Central door lock,Driver-side airbag,Elect...",NaN,Germany,4.1,4.6,3.8


In [5]:
df.shape

(15919, 36)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   short_description       15873 non-null  object 
 1   body_type               15859 non-null  object 
 2   price_€                 15919 non-null  float64
 3   vat                     11406 non-null  object 
 4   km                      14895 non-null  float64
 5   registration            14322 non-null  float64
 6   prev_owner              9091 non-null   float64
 7   hp_kW                   15831 non-null  float64
 8   type                    15917 non-null  object 
 9   previous_owners         9279 non-null   float64
 10  warranty                4853 non-null   float64
 11  make                    15919 non-null  object 
 12  model                   15919 non-null  object 
 13  body_color              15322 non-null  object 
 14  paint_type              10147 non-null

In [7]:
df.isnull().sum()

short_description            46
body_type                    60
price_€                       0
vat                        4513
km                         1024
registration               1597
prev_owner                 6828
hp_kW                        88
type                          2
previous_owners            6640
warranty                  11066
make                          0
model                         0
body_color                  597
paint_type                 5772
body_color_original        3774
upholstery                 3720
nr_of_doors                 212
nr_of_seats                 977
gearing_type                  0
engine_displacement_cc      496
cylinders                  5680
weight_kg                  6974
drive_chain                6858
fuel                          0
co2e_g                     2442
emission_class             3628
comfort&convenience         920
entertainment&media        1374
extras                     2962
safety&security             982
gears   

In [8]:
#df.duplicated().sum()

In [9]:
#df.drop_duplicates(keep="first", inplace=True, ignore_index=True)

In [10]:
df.isnull().sum()

short_description            46
body_type                    60
price_€                       0
vat                        4513
km                         1024
registration               1597
prev_owner                 6828
hp_kW                        88
type                          2
previous_owners            6640
warranty                  11066
make                          0
model                         0
body_color                  597
paint_type                 5772
body_color_original        3774
upholstery                 3720
nr_of_doors                 212
nr_of_seats                 977
gearing_type                  0
engine_displacement_cc      496
cylinders                  5680
weight_kg                  6974
drive_chain                6858
fuel                          0
co2e_g                     2442
emission_class             3628
comfort&convenience         920
entertainment&media        1374
extras                     2962
safety&security             982
gears   

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price_€,15919.0,18019.896727,7386.169409,13.0,12850.0,16900.0,21900.0,74600.0
km,14895.0,34130.127828,37352.977568,0.0,5153.0,22740.0,49371.5,317000.0
registration,14322.0,2017.457618,1.076917,2016.0,2016.0,2018.0,2018.0,2019.0
prev_owner,9091.0,1.089979,0.294877,1.0,1.0,1.0,1.0,4.0
hp_kW,15831.0,88.367696,26.835135,1.0,66.0,85.0,101.0,294.0
previous_owners,9279.0,1.067895,0.329812,0.0,1.0,1.0,1.0,4.0
warranty,4853.0,21.927261,15.771120,1.0,12.0,12.0,24.0,72.0
nr_of_doors,15707.0,4.656077,0.645298,1.0,4.0,5.0,5.0,7.0
nr_of_seats,14942.0,4.949873,0.489872,2.0,5.0,5.0,5.0,7.0
engine_displacement_cc,15423.0,1423.540881,333.530658,1.0,1229.0,1461.0,1598.0,16000.0


In [12]:
df.describe(include="O").T

,count,unique,top,freq
short_description,15873,10001,SPB 1.6 TDI 116 CV S tronic Sport,64
body_type,15859,9,Sedans,7903
vat,11406,2,VAT deductible,10980
type,15917,5,Used,11096
make,15919,3,Opel,7343
model,15919,9,A3,3097
body_color,15322,14,Black,3745
paint_type,10147,3,Metallic,9794
body_color_original,12145,1926,Onyx Schwarz,338
upholstery,12199,6,Cloth,9206


In [13]:
df["price_€"].sort_values()

8594        13.0
8828       120.0
6066       255.0
8829       331.0
8827      4950.0
          ...   
15828    64332.0
3587     64900.0
3649     67600.0
15826    68320.0
3648     74600.0
Name: price_€, Length: 15919, dtype: float64

In [14]:
df["age"] = 2019 - df.registration

In [15]:
df.drop(columns=["registration", "short_description"], inplace=True)

In [16]:
# we created age column we don't need to registiration column anymore. Also we don't need short_description too we droped.

In [17]:
def check_df(col):
    print('column name : ', col)
    print('--------------------------------')
    print('Per_of_Nulls   : ', '%', round(df[col].isnull().sum() / df.shape[0]*100, 2))
    print('Num_of_Nulls   : ', df[col].isnull().sum())
    print(df[col].value_counts(dropna = False))
    print("*"*30)
    print("\n\n")

## body_type

In [18]:
check_df("body_type")

column name :  body_type
--------------------------------
Per_of_Nulls   :  % 0.38
Num_of_Nulls   :  60
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64
******************************





In [19]:
df[df.body_type.isnull()]

,body_type,price_€,vat,km,prev_owner,hp_kW,type,previous_owners,warranty,make,model,body_color,paint_type,body_color_original,upholstery,nr_of_doors,nr_of_seats,gearing_type,engine_displacement_cc,cylinders,weight_kg,drive_chain,fuel,co2e_g,emission_class,comfort&convenience,entertainment&media,extras,safety&security,gears,country_version,consumption_comb,consumption_city,consumption_country,age
3175,NaN,14400.0,NaN,119000.0,1.0,81.0,Used,1.0,NaN,Audi,A3,Black,Metallic,NaN,Cloth,5.0,5.0,Manual,1598.0,4.0,1385.0,NaN,Diesel,99.0,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport package,Trailer hitch,Voice...","ABS,Adaptive headlights,Central door lock,Cent...",6.0,NaN,3.8,4.5,3.4,3.0
3255,NaN,18500.0,Price negotiable,70000.0,NaN,81.0,Used,NaN,NaN,Audi,A3,White,NaN,NaN,Cloth,3.0,5.0,Automatic,1598.0,4.0,1255.0,NaN,Diesel,99.0,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,On-board computer,Radio,USB","Alloy wheels,Shift paddles","ABS,Alarm system,Central door lock,Driver-side...",7.0,NaN,3.8,4.3,3.5,3.0
3975,NaN,19950.0,NaN,25000.0,2.0,85.0,Used,2.0,NaN,Audi,A3,Blue,Metallic,NaN,NaN,5.0,5.0,Automatic,999.0,3.0,1275.0,NaN,Benzine,107.0,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Digital radio,MP3,On-board...","Alloy wheels,Roof rack,Sport package,Sport sea...","ABS,Adaptive headlights,Central door lock with...",7.0,NaN,4.7,5.6,4.2,2.0
3997,NaN,19750.0,VAT deductible,38800.0,1.0,82.0,Used,1.0,6.0,Audi,A3,Black,NaN,NOIR,NaN,3.0,NaN,Automatic,1598.0,NaN,NaN,NaN,Diesel,NaN,NaN,"Air conditioning,Automatic climate control,Cru...","Bluetooth,On-board computer,Radio",Alloy wheels,"ABS,Daytime running lights,Driver-side airbag,...",NaN,NaN,NaN,NaN,NaN,2.0
4206,NaN,18490.0,NaN,20000.0,NaN,92.0,Used,NaN,NaN,Audi,A3,White,NaN,NaN,NaN,5.0,NaN,Manual,NaN,NaN,NaN,NaN,Benzine,117.0,NaN,Air conditioning,NaN,NaN,"ABS,Central door lock,Daytime running lights,D...",NaN,NaN,5.1,6.7,4.1,2.0
4297,NaN,17990.0,NaN,20000.0,NaN,92.0,Used,NaN,NaN,Audi,A3,White,NaN,NaN,NaN,5.0,NaN,Manual,NaN,NaN,NaN,NaN,Benzine,117.0,NaN,Air conditioning,NaN,NaN,"ABS,Central door lock,Daytime running lights,D...",NaN,NaN,5.1,6.7,4.1,2.0
4298,NaN,17990.0,NaN,20000.0,NaN,92.0,Used,NaN,NaN,Audi,A3,White,NaN,NaN,NaN,5.0,NaN,Manual,NaN,NaN,NaN,NaN,Benzine,117.0,NaN,Air conditioning,NaN,NaN,"ABS,Central door lock,Daytime running lights,D...",NaN,NaN,5.1,6.7,4.1,2.0
5718,NaN,12990.0,NaN,47000.0,NaN,100.0,Used,NaN,NaN,Opel,Astra,Black,NaN,NaN,NaN,5.0,NaN,Automatic,NaN,NaN,NaN,NaN,Diesel,119.0,NaN,"Air conditioning,Multi-function steering wheel",NaN,NaN,"ABS,Central door lock,Daytime running lights,D...",NaN,NaN,4.5,5.7,3.8,3.0
5938,NaN,12990.0,NaN,47000.0,NaN,100.0,Used,NaN,NaN,Opel,Astra,Black,NaN,NaN,NaN,5.0,NaN,Automatic,NaN,NaN,NaN,NaN,Diesel,119.0,NaN,"Air conditioning,Multi-function steering wheel",NaN,NaN,"ABS,Central door lock,Daytime running lights,D...",NaN,NaN,4.5,5.7,3.8,3.0
5940,NaN,12990.0,NaN,47000.0,NaN,100.0,Used,NaN,NaN,Opel,Astra,Black,NaN,NaN,NaN,5.0,NaN,Automatic,NaN,NaN,NaN,NaN,Diesel,119.0,NaN,"Air conditioning,Multi-function steering wheel",NaN,NaN,"ABS,Central door lock,Daytime running lights,D...",NaN,NaN,4.5,5.7,3.8,3.0


In [20]:
df.groupby(["make", "model"])["body_type"].value_counts(dropna=False)

make     model     body_type    
Audi     A1        Sedans           1538
                   Compact          1039
                   Station wagon      21
                   Other              13
                   Coupe               2
                   Van                 1
         A2        Off-Road            1
         A3        Sedans           2598
                   Station wagon     282
                   Compact           182
                   Other              16
                   Convertible         8
                   NaN                 7
                   Coupe               4
Opel     Astra     Station wagon    1211
                   Sedans           1053
                   Compact           185
                   Other              67
                   NaN                 7
                   Coupe               2
                   Off-Road            1
         Corsa     Compact          1230
                   Sedans            875
                   Other

In [21]:
df.groupby(["make", "model", "nr_of_doors"])["body_type"].value_counts(dropna=False)

make     model   nr_of_doors  body_type
Audi     A1      2.0          Compact       69
                              Sedans        29
                              Coupe          1
                 3.0          Sedans       130
                              Compact       80
                                          ... 
Renault  Espace  5.0          Other          7
                              Off-Road       6
                              Compact        5
                              Coupe          1
                 7.0          Van            1
Name: body_type, Length: 111, dtype: int64

In [22]:
#df.groupby(["make", "model"])["body_type"].describe(include="object")["top"]
df.groupby(["make", "model"])["body_type"].apply(lambda x: x.mode()[0])

make     model   
Audi     A1                 Sedans
         A2               Off-Road
         A3                 Sedans
Opel     Astra       Station wagon
         Corsa             Compact
         Insignia    Station wagon
Renault  Clio               Sedans
         Duster           Off-Road
         Espace                Van
Name: body_type, dtype: object

In [23]:
df.body_type = df.groupby(["make", "model"])["body_type"].apply(lambda x: x.fillna(x.mode()[0]))

In [24]:
df.body_type.value_counts()

Sedans           7925
Station wagon    3563
Compact          3155
Van               809
Other             290
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [25]:
df.shape

(15919, 35)

## vat

In [26]:
check_df("vat")

column name :  vat
--------------------------------
Per_of_Nulls   :  % 28.35
Num_of_Nulls   :  4513
VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64
******************************





In [27]:
df.groupby("vat").describe(include="O")

body_type                        type                     \
                     count unique     top  freq  count unique   top  freq   
vat                                                                         
Price negotiable       426      9  Sedans   286    426      3  Used   421   
VAT deductible       10980      9  Sedans  5248  10980      5  Used  6787   

                   make                     model                  body_color  \
                  count unique   top  freq  count unique top  freq      count   
vat                                                                             
Price negotiable    426      3  Audi   182    426      7  A3   101        426   
VAT deductible    10980      3  Opel  5415  10980      9  A3  2209      10586   

                                     paint_type                         \
                 unique    top  freq      count unique       top  freq   
vat                                                                      
Price negotiable     10  Black   109        245      1  Metallic   245   
VAT deductible       13  Black  2704       7558      3  Metallic  7312   

                 body_color_original                                         \
                               count unique                        top freq   
vat                                                                           
Price negotiable                  17     17  Floretzilver (grijs metal    1   
VAT deductible                  9543   1568               Onyx Schwarz  314   

                 upholstery                     gearing_type                 \
                      count unique    top  freq        count unique     top   
vat                                                                           
Price negotiable        134      4  Cloth    82          426      3  Manual   
VAT deductible         9780      6  Cloth  7404        10980      3  Manual   

                       drive_chain                       fuel                  \
                  freq       count unique    top  freq  count unique      top   
vat                                                                             
Price negotiable   204          95      2  front    92    426      4   Diesel   
VAT deductible    5502        6876      2  front  6762  10980      4  Benzine   

                       emission_class                       \
                  freq          count unique     top  freq   
vat                                                          
Price negotiable   223            401      6  Euro 6   361   
VAT deductible    6183           8801      6  Euro 6  6915   

                 comfort&convenience         \
                               count unique   
vat                                           
Price negotiable                 423    333   
VAT deductible                 10811   4368   

                                                                          \
                                                                top freq   
vat                                                                        
Price negotiable  Air conditioning,Seat ventilation,Split rear s...   15   
VAT deductible    Air conditioning,Electrical side mirrors,Hill ...  163   

                 entertainment&media         \
                               count unique   
vat                                           
Price negotiable                 414    100   
VAT deductible                 10609    300   

                                                                           \
                                                                top  freq   
vat                                                                         
Price negotiable  Bluetooth,CD player,Hands-free equipment,MP3,O...    32   
VAT deductible    Bluetooth,Hands-free equipment,On-board comput...  1088   

                 extras                            safety&security         \
                  count uniqu

In [28]:
df.vat.value_counts().index

Index(['VAT deductible', 'Price negotiable'], dtype='object')

In [29]:
df.vat = df.vat.fillna(method="bfill")

In [30]:
df.vat.value_counts(dropna=False)

VAT deductible      15083
Price negotiable      836
Name: vat, dtype: int64

## type

In [31]:
df.type.mode()

0    Used
dtype: object

In [32]:
# there are onşy 2 missing value in this column so we will fill these values with ffill method
df.type = df.type.fillna(method="ffill")

In [33]:
df.type.isnull().sum()

0

## age 

In [34]:
df.age.isnull().sum()

1597

In [35]:
df.groupby("type").age.median()

type
Demonstration     0.0
Employee's car    1.0
New               0.0
Pre-registered    0.0
Used              2.0
Name: age, dtype: float64

In [36]:
df["age"] = df.groupby("type").age.apply(lambda x: x.fillna(x.median()))

In [37]:
df.age.value_counts()

1.0    4525
0.0    4412
3.0    3674
2.0    3308
Name: age, dtype: int64

In [38]:
df.age. isnull().sum()

0

## km

In [39]:
check_df("km")

column name :  km
--------------------------------
Per_of_Nulls   :  % 6.43
Num_of_Nulls   :  1024
10.0       1045
NaN        1024
1.0         367
5.0         170
50.0        148
           ... 
67469.0       1
43197.0       1
10027.0       1
35882.0       1
57.0          1
Name: km, Length: 6690, dtype: int64
******************************





In [40]:
df.groupby(["type","age"])["km"].mean()

type            age
Demonstration   0.0     3373.867550
                1.0     8294.200000
                2.0    25327.296296
                3.0    45500.000000
Employee's car  0.0     4473.933884
                1.0    19295.278068
                2.0    37926.505051
                3.0    59123.647059
New             0.0       16.987484
                1.0       53.500000
Pre-registered  0.0      100.648820
                1.0      435.514925
                2.0     1854.000000
                3.0       10.000000
Used            0.0     3712.170610
                1.0    18807.503474
                2.0    41887.937559
                3.0    77566.423098
Name: km, dtype: float64

In [41]:
df.km = df.groupby(["type","age"])["km"].apply(lambda x: x.fillna(x.mean()))

In [42]:
df.km.isnull().sum()

0

## prev_owner and previous_owners

In [43]:
check_df("prev_owner")

column name :  prev_owner
--------------------------------
Per_of_Nulls   :  % 42.89
Num_of_Nulls   :  6828
1.0    8294
NaN    6828
2.0     778
3.0      17
4.0       2
Name: prev_owner, dtype: int64
******************************





In [44]:
check_df("previous_owners")

column name :  previous_owners
--------------------------------
Per_of_Nulls   :  % 41.71
Num_of_Nulls   :  6640
1.0    8294
NaN    6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: previous_owners, dtype: int64
******************************





In [45]:
df.groupby("previous_owners")["prev_owner"].value_counts(dropna=False)

previous_owners  prev_owner
0.0              NaN            188
1.0              1.0           8294
2.0              2.0            778
3.0              3.0             17
4.0              4.0              2
Name: prev_owner, dtype: int64

In [46]:
# as we see prev_owner and previous_owners columns are the same 
df.drop(columns="prev_owner", inplace=True)

In [47]:
df.groupby("age")["previous_owners"].apply(lambda x: x.fillna(x.mode()[0])).value_counts()

1.0    14934
2.0      778
0.0      188
3.0       17
4.0        2
Name: previous_owners, dtype: int64

In [48]:
df.groupby("type")["previous_owners"].apply(lambda x: x.fillna(x.mode()[0])).value_counts()

1.0    13463
0.0     1659
2.0      778
3.0       17
4.0        2
Name: previous_owners, dtype: int64

In [49]:
df.previous_owners = df.groupby("type")["previous_owners"].apply(lambda x: x.fillna(x.mode()[0]))

In [50]:
df.previous_owners.isnull().sum()

0

In [51]:
df.previous_owners.value_counts(dropna=False)

1.0    13463
0.0     1659
2.0      778
3.0       17
4.0        2
Name: previous_owners, dtype: int64

## hp_kW

In [52]:
check_df("hp_kW")

column name :  hp_kW
--------------------------------
Per_of_Nulls   :  % 0.55
Num_of_Nulls   :  88
85.0     2542
66.0     2122
81.0     1402
100.0    1308
110.0    1112
         ... 
84.0        1
195.0       1
44.0        1
239.0       1
9.0         1
Name: hp_kW, Length: 81, dtype: int64
******************************





In [53]:
df.groupby(["make", "model","body_type"])["hp_kW"].median()

make     model     body_type    
Audi     A1        Compact           70.0
                   Coupe             77.5
                   Other             70.0
                   Sedans            70.0
                   Station wagon     70.0
                   Van              147.0
         A2        Off-Road          85.0
         A3        Compact           85.0
                   Convertible      110.0
                   Coupe             81.0
                   Other             85.0
                   Sedans            85.0
                   Station wagon     85.0
Opel     Astra     Compact           81.0
                   Coupe            106.5
                   Off-Road         110.0
                   Other             81.0
                   Sedans            81.0
                   Station wagon    100.0
         Corsa     Compact           66.0
                   Coupe             55.0
                   Off-Road          66.0
                   Other             66.0
 

In [54]:
df.hp_kW = df.groupby(["make", "model","body_type"])["hp_kW"].apply(lambda x : x.fillna(x.median()))

## warranty    

In [55]:
check_df("warranty")

column name :  warranty
--------------------------------
Per_of_Nulls   :  % 69.51
Num_of_Nulls   :  11066
NaN     11066
12.0     2594
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
50.0        4
26.0        4
16.0        4
4.0         3
1.0         3
19.0        3
34.0        3
13.0        3
28.0        2
22.0        2
14.0        2
11.0        2
46.0        2
21.0        2
9.0         2
17.0        2
45.0        2
33.0        1
40.0        1
65.0        1
10.0        1
15.0        1
7.0         1
8.0         1
56.0        1
49.0        1
47.0        1
30.0        1
Name: warranty, dtype: int64
******************************





In [56]:
df.groupby(["make", "model","age"])["warranty"].mean()

make     model     age
Audi     A1        0.0    27.970874
                   1.0    26.300429
                   2.0    17.679389
                   3.0    13.926174
         A2        1.0          NaN
         A3        0.0    27.697211
                   1.0    21.706897
                   2.0    16.654709
                   3.0    17.111511
Opel     Astra     0.0    37.803571
                   1.0    25.795455
                   2.0    14.987952
                   3.0    24.797468
         Corsa     0.0    27.472393
                   1.0    17.355140
                   2.0    22.230769
                   3.0    27.060811
         Insignia  0.0    31.862637
                   1.0    25.347826
                   2.0    17.057971
                   3.0    15.225434
Renault  Clio      0.0    35.510204
                   1.0    15.686441
                   2.0    16.733333
                   3.0    10.864198
         Duster    0.0    12.000000
         Espace    0.0    37.226415
     

In [57]:
df.groupby(["make", "model","age"])["warranty"].median()

make     model     age
Audi     A1        0.0    24.0
                   1.0    24.0
                   2.0    12.0
                   3.0    12.0
         A2        1.0     NaN
         A3        0.0    24.0
                   1.0    24.0
                   2.0    12.0
                   3.0    12.0
Opel     Astra     0.0    24.0
                   1.0    12.0
                   2.0    12.0
                   3.0    12.0
         Corsa     0.0    24.0
                   1.0    12.0
                   2.0    12.0
                   3.0    12.0
         Insignia  0.0    24.0
                   1.0    12.0
                   2.0    12.0
                   3.0    12.0
Renault  Clio      0.0    24.0
                   1.0    12.0
                   2.0    12.0
                   3.0    12.0
         Duster    0.0    12.0
         Espace    0.0    36.0
                   1.0    22.0
                   2.0    12.0
                   3.0    12.0
Name: warranty, dtype: float64

In [58]:
# df[df.type == "Used"]["warranty"].replace(np.nan, 0, inplace=True)
# df[df.type == "Used"]["warranty"].fillna(0)

In [59]:
df.groupby(["type", "age"])["warranty"].value_counts(dropna=False)

type           age  warranty
Demonstration  0.0  NaN         482
                    24.0         46
                    48.0         27
                    12.0         23
                    60.0         22
                               ... 
Used           3.0  21.0          1
                    23.0          1
                    26.0          1
                    40.0          1
                    48.0          1
Name: warranty, Length: 152, dtype: int64

In [60]:
df["warranty"].value_counts(dropna=False)

NaN     11066
12.0     2594
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
50.0        4
26.0        4
16.0        4
4.0         3
1.0         3
19.0        3
34.0        3
13.0        3
28.0        2
22.0        2
14.0        2
11.0        2
46.0        2
21.0        2
9.0         2
17.0        2
45.0        2
33.0        1
40.0        1
65.0        1
10.0        1
15.0        1
7.0         1
8.0         1
56.0        1
49.0        1
47.0        1
30.0        1
Name: warranty, dtype: int64

In [61]:
df["warranty"] = df["warranty"].replace(to_replace=np.nan, value=0).astype("float")

In [62]:
df.loc[df["warranty"] != 0, "warranty"] = 1

In [63]:
df["warranty"].value_counts(dropna=False)

0.0    11066
1.0     4853
Name: warranty, dtype: int64

## make and model

In [64]:
check_df("make")

column name :  make
--------------------------------
Per_of_Nulls   :  % 0.0
Num_of_Nulls   :  0
Opel       7343
Audi       5712
Renault    2864
Name: make, dtype: int64
******************************





In [65]:
check_df("model")

column name :  model
--------------------------------
Per_of_Nulls   :  % 0.0
Num_of_Nulls   :  0
A3          3097
A1          2614
Insignia    2598
Astra       2526
Corsa       2219
Clio        1839
Espace       991
Duster        34
A2             1
Name: model, dtype: int64
******************************





In [66]:
# There is no null values in make and model columns

## body_color  and body_color_original             

In [67]:
check_df("body_color")

column name :  body_color
--------------------------------
Per_of_Nulls   :  % 3.75
Num_of_Nulls   :  597
Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: body_color, dtype: int64
******************************





In [68]:
check_df("body_color_original")

column name :  body_color_original
--------------------------------
Per_of_Nulls   :  % 23.71
Num_of_Nulls   :  3774
NaN                              3774
Onyx Schwarz                      338
Bianco                            282
Mythosschwarz Metallic            238
Brillantschwarz                   216
                                 ... 
Rouge-Braun (G0Y)                   1
VARI COLRI DISPONIBILI              1
Kokosnussbraun Metallic             1
Farbe frei wählbar                  1
Perlmutt-Weiß Metallic (Weiß)       1
Name: body_color_original, Length: 1927, dtype: int64
******************************





In [69]:
df[df.body_color.isnull()]["body_color_original"].value_counts()

wählbar - ggf. mit Aufpreis       118
wählbar                            88
Metallic o. Uni (wählbar)          33
wählbar - ggf gegen Aufpreis       25
Other                              14
                                 ... 
rouge eclat                         1
quarzgrau dunkel Grau metallic      1
Licht Grau M2                       1
graphitgrau Metallik                1
Farbe: Sonstige                     1
Name: body_color_original, Length: 85, dtype: int64

In [70]:
df.body_color = df.body_color.fillna(method="bfill")

In [71]:
df.body_color.isnull().sum()

0

## paint_type

In [72]:
check_df("paint_type")

column name :  paint_type
--------------------------------
Per_of_Nulls   :  % 36.26
Num_of_Nulls   :  5772
Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: paint_type, dtype: int64
******************************





In [73]:
df.groupby("paint_type")["body_color_original"].value_counts()

paint_type  body_color_original      
Metallic    Onyx Schwarz                 290
            Mythosschwarz Metallic       225
            Quarz Grau                   192
            Gris                         137
            Nero                         115
                                        ... 
Uni/basic   VERDE MARE PASTELLO (LT3)      1
            WEISS 'NACRE'                  1
            Weiss                          1
            bleu                           1
            grigio monsone                 1
Name: body_color_original, Length: 1571, dtype: int64

In [74]:
df[df.paint_type.isnull()]["body_color_original"].value_counts()

Brillantschwarz                179
Bianco                         163
wählbar - ggf. mit Aufpreis    118
Blanco                         110
Grigio                          91
                              ... 
schnee-weiss                     1
schneeweiß/schwarz               1
rouge eclat                      1
Gris Cosmique                    1
Cosmos Blau Met                  1
Name: body_color_original, Length: 623, dtype: int64

In [75]:
df["paint_type"] = df["paint_type"].fillna(method="ffill")

In [76]:
df.paint_type.isnull().sum()

0

## body_color_original 

In [77]:
# Since this column contains irrelevant info and its languages is foreign language, we've dropped this column.

In [78]:
df.drop(columns="body_color_original", inplace=True)

## upholstery   

In [79]:
check_df("upholstery")

column name :  upholstery
--------------------------------
Per_of_Nulls   :  % 23.37
Num_of_Nulls   :  3720
Cloth           9206
NaN             3720
Part leather    1499
Full leather    1009
Other            368
Velour            60
alcantara         57
Name: upholstery, dtype: int64
******************************





In [80]:
df.extras.value_counts().head(50)

Alloy wheels                                                                                        3245
Alloy wheels,Touch screen                                                                            697
Alloy wheels,Voice Control                                                                           577
Alloy wheels,Touch screen,Voice Control                                                              541
Alloy wheels,Roof rack                                                                               385
Alloy wheels,Sport seats                                                                             380
Roof rack                                                                                            337
Alloy wheels,Catalytic Converter                                                                     271
Alloy wheels,Sport seats,Sport suspension                                                            234
Alloy wheels,Catalytic Converter,Touch screen          

In [81]:
get_diff_category_column(df["comfort&convenience"])

{0: 'Air conditioning',
 1: 'Air suspensio',
 2: 'Armrest',
 3: 'Automatic climate control',
 4: 'Auxiliary heating',
 5: 'Cruise control',
 6: 'Electric Starter',
 7: 'Electric tailgate',
 8: 'Electrical side mirrors',
 9: 'Electrically adjustable seats',
 10: 'Electrically heated windshield',
 11: 'Heads-up display',
 12: 'Heated steering wheel',
 13: 'Hill Holder',
 14: 'Keyless central door lock',
 15: 'Leather seats',
 16: 'Leather steering wheel',
 17: 'Light sensor',
 18: 'Lumbar support',
 19: 'Massage seats',
 20: 'Multi-function steering wheel',
 21: 'Navigation system',
 22: 'Panorama roof',
 23: 'Park Distance Control',
 24: 'Parking assist system camera',
 25: 'Parking assist system self-steering',
 26: 'Parking assist system sensors front',
 27: 'Parking assist system sensors rear',
 28: 'Power windows',
 29: 'Rain sensor',
 30: 'Seat heating',
 31: 'Seat ventilatio',
 32: 'Split rear seats',
 33: 'Start-stop system',
 34: 'Sunroof',
 35: 'Tinted windows',
 36: 'Wind defl

In [82]:
df["comfort&convenience"].str.contains("Leather seats").value_counts()

False    14951
True        48
Name: comfort&convenience, dtype: int64

In [83]:
df[df.upholstery.isnull()]["comfort&convenience"].str.contains("Leather seats").value_counts(dropna=False)

False    2939
NaN       779
True        2
Name: comfort&convenience, dtype: int64

In [84]:
df.groupby(["make", "model", "price_€"])["upholstery"].fillna(method="pad").fillna(method="backfill").value_counts(dropna=False, normalize=True)

Cloth           0.748288
Part leather    0.124003
Full leather    0.085181
Other           0.031849
Velour          0.006031
alcantara       0.004649
Name: upholstery, dtype: float64

In [85]:
df.upholstery = df.groupby(["make", "model", "price_€"])["upholstery"].fillna(method="pad").fillna(method="backfill")

In [86]:
# As the second option we can create an additional column according to "price_€" column("high", "medium", "low" price).
# We can fill NaN values of upholstery column by grouping this new column.

## nr_of_doors             

In [87]:
check_df("nr_of_doors")

column name :  nr_of_doors
--------------------------------
Per_of_Nulls   :  % 1.33
Num_of_Nulls   :  212
5.0    11575
4.0     3079
3.0      832
2.0      219
NaN      212
1.0        1
7.0        1
Name: nr_of_doors, dtype: int64
******************************





In [88]:
df[(df.make == "Opel")&(df.model == "Corsa")&(df.body_type == "Van")]["nr_of_doors"].fillna(5, inplace=True)

In [89]:
df.groupby(["make", "model", "body_type"])["nr_of_doors"].value_counts(dropna = False)

make     model   body_type    nr_of_doors
Audi     A1      Compact      5.0            666
                              4.0            207
                              3.0             80
                              2.0             69
                              NaN             17
                                            ... 
Renault  Espace  Transporter  5.0             52
                 Van          5.0            704
                              4.0             90
                              NaN              8
                              7.0              1
Name: nr_of_doors, Length: 129, dtype: int64

In [90]:
df.nr_of_doors = df.groupby(["make","model", "body_type"])['nr_of_doors'].apply(lambda x: x.fillna(x.mode()[0] if any(x.mode()) else df['nr_of_doors'].mode()[0]))

In [91]:
df.nr_of_doors.isnull().sum()

0

## nr_of_seats

In [92]:
check_df("nr_of_seats")

column name :  nr_of_seats
--------------------------------
Per_of_Nulls   :  % 6.14
Num_of_Nulls   :  977
5.0    13336
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: nr_of_seats, dtype: int64
******************************





In [93]:
df.groupby(["make", "model", "body_type"])["nr_of_seats"].value_counts(dropna = False)

make     model   body_type    nr_of_seats
Audi     A1      Compact      5.0            538
                              4.0            431
                              NaN             68
                              3.0              1
                              6.0              1
                                            ... 
Renault  Espace  Transporter  NaN              1
                 Van          5.0            383
                              7.0            315
                              NaN            104
                              4.0              1
Name: nr_of_seats, Length: 116, dtype: int64

In [94]:
df.nr_of_seats = df.groupby(["make","model", "body_type"])['nr_of_seats'].apply(lambda x: x.fillna(x.mode()[0] if any(x.mode()) else df['nr_of_seats'].mode()[0]))

In [95]:
df.nr_of_seats.isnull().sum()

0

## gearing_type            

In [96]:
check_df("gearing_type")

column name :  gearing_type
--------------------------------
Per_of_Nulls   :  % 0.0
Num_of_Nulls   :  0
Manual            8153
Automatic         7297
Semi-automatic     469
Name: gearing_type, dtype: int64
******************************





In [97]:
# There is no null value to fill

## cylinders

In [98]:
check_df("cylinders")

column name :  cylinders
--------------------------------
Per_of_Nulls   :  % 35.68
Num_of_Nulls   :  5680
4.0    8105
NaN    5680
3.0    2104
5.0      22
6.0       3
8.0       2
2.0       2
1.0       1
Name: cylinders, dtype: int64
******************************





In [99]:
df.groupby(["make", "model", "hp_kW"])["cylinders"].apply(lambda x: x.fillna(x.mode()[0] if any(x.mode()) else df["cylinders"].mode()[0])).value_counts(dropna=False)

4.0    12559
3.0     3325
5.0       22
8.0        7
6.0        3
2.0        2
1.0        1
Name: cylinders, dtype: int64

In [100]:
df["cylinders"] = df.groupby(["make", "model", "hp_kW"])["cylinders"].apply(lambda x: x.fillna(x.mode()[0] if any(x.mode()) else df["cylinders"].mode()[0]))

In [101]:
df.cylinders.isnull().sum()

0

## engine_displacement_cc

In [102]:
check_df("engine_displacement_cc")

column name :  engine_displacement_cc
--------------------------------
Per_of_Nulls   :  % 3.12
Num_of_Nulls   :  496
1598.0    4761
999.0     2438
1398.0    1314
1399.0     749
1229.0     677
          ... 
122.0        1
1198.0       1
1195.0       1
2967.0       1
1800.0       1
Name: engine_displacement_cc, Length: 78, dtype: int64
******************************





In [103]:
df["engine_displacement_cc"] = df.groupby(["make","model", "cylinders", "hp_kW"])["engine_displacement_cc"].apply(lambda x : x.fillna(x.mode()[0] if any(x.mode()) else df.engine_displacement_cc.mode()[0]))

In [104]:
df.engine_displacement_cc.isnull().sum()

0

## weight_kg               

In [105]:
check_df("weight_kg")

column name :  weight_kg
--------------------------------
Per_of_Nulls   :  % 43.81
Num_of_Nulls   :  6974
NaN       6974
1163.0     574
1360.0     356
1165.0     301
1335.0     242
          ... 
1960.0       1
1258.0       1
1167.0       1
1331.0       1
2037.0       1
Name: weight_kg, Length: 435, dtype: int64
******************************





In [106]:
df.groupby(["make", "model", "body_type"])["weight_kg"].value_counts(dropna=False).head(60)

make  model  body_type  weight_kg
Audi  A1     Compact    NaN          526
                        1180.0        86
                        1165.0        77
                        1200.0        65
                        1135.0        53
                        1195.0        40
                        1065.0        36
                        1110.0        22
                        1250.0        17
                        1105.0        12
                        1090.0         9
                        1215.0         9
                        1240.0         7
                        1035.0         6
                        1134.0         6
                        1155.0         5
                        1205.0         5
                        1275.0         5
                        1485.0         5
                        1225.0         4
                        1600.0         4
                        1115.0         3
                        1133.0         3
                       

In [107]:
df.groupby(["make", "model", "body_type"])["weight_kg"].apply(lambda x : x.fillna(x.mean())).value_counts(dropna=False).head(100)                                   

1395.619545    978
1627.555267    845
1174.057087    724
1403.741611    622
1259.656216    613
              ... 
1215.000000     29
1633.000000     29
1322.000000     29
1385.000000     28
1155.000000     27
Name: weight_kg, Length: 100, dtype: int64

In [108]:
# df.weight_kg = df.groupby(["make", "model", "body_type"])["weight_kg"].apply(lambda x : x.fillna(x.mean() if any(x.mean()) else df.weight_kg.mean())).value_counts(dropna=False) 

In [109]:
df.weight_kg.isnull().sum()

6974

In [110]:
df.weight_kg.mean()

1351.1065399664617

In [111]:
df.weight_kg.median()

1288.0

In [112]:
# since there are much extreme values, we will fill with median

In [113]:
df["weight_kg"] = df.groupby(["make", "model", "body_type"])["weight_kg"].apply(lambda x : x.fillna(x.median() if (x.median() >= 1) else df["weight_kg"].median()))

In [114]:
df.weight_kg.isnull().sum()

0

## drive_chain

In [115]:
check_df("drive_chain")

column name :  drive_chain
--------------------------------
Per_of_Nulls   :  % 43.08
Num_of_Nulls   :  6858
front    8886
NaN      6858
4WD       171
rear        4
Name: drive_chain, dtype: int64
******************************





In [116]:
df.groupby(["make", "model", "body_type"])["drive_chain"].value_counts(dropna=False)

make     model   body_type    drive_chain
Audi     A1      Compact      front          685
                              NaN            352
                              4WD              2
                 Coupe        NaN              2
                 Other        front           12
                                            ... 
Renault  Espace  Transporter  front            6
                              4WD              1
                 Van          NaN            391
                              front          374
                              4WD             38
Name: drive_chain, Length: 107, dtype: int64

In [117]:
df.drive_chain = df.groupby(["make", "model", "body_type"])["drive_chain"].apply(lambda x : x.fillna(x.mode()[0] if any(x.mode()) else df["drive_chain"].mode()[0]))

In [118]:
df.drive_chain.value_counts()

front    15711
4WD        204
rear         4
Name: drive_chain, dtype: int64

In [119]:
df.drive_chain.isnull().sum()

0

## co2e_g                  

In [120]:
check_df("co2e_g")

column name :  co2e_g
--------------------------------
Per_of_Nulls   :  % 15.34
Num_of_Nulls   :  2442
NaN       2442
120.0      740
99.0       545
97.0       537
104.0      501
          ... 
160.0        1
331.0        1
990.0        1
1060.0       1
193.0        1
Name: co2e_g, Length: 120, dtype: int64
******************************





In [121]:
df.groupby(["make", "model", "fuel"])["co2e_g"].value_counts(dropna=False)

make     model   fuel     co2e_g
Audi     A1      Benzine  97.0      287
                          NaN       238
                          102.0     229
                          108.0     195
                          111.0     181
                                   ... 
Renault  Espace  Diesel   191.0       2
                          117.0       1
                          122.0       1
                          140.0       1
                          174.0       1
Name: co2e_g, Length: 516, dtype: int64

In [122]:
df.co2e_g.mean()

123.79357423758997

In [123]:
df.co2e_g.median()

116.0

In [124]:
df.co2e_g.sort_values()

8110     14.0
3617     36.0
3612     36.0
3615     36.0
12547    45.0
         ... 
15883     NaN
15903     NaN
15906     NaN
15908     NaN
15909     NaN
Name: co2e_g, Length: 15919, dtype: float64

In [125]:
df.co2e_g = df.groupby(["make", "model", "fuel"])["co2e_g"].apply(lambda x : x.fillna(x.median() if (x.median() > 13) else df["co2e_g"].median()))

## emission_class

In [126]:
check_df("emission_class")

column name :  emission_class
--------------------------------
Per_of_Nulls   :  % 22.79
Num_of_Nulls   :  3628
Euro 6          10139
NaN              3628
Euro 6d-TEMP     1845
Euro 6c           127
Euro 5             78
Euro 6d            62
Euro 4             40
Name: emission_class, dtype: int64
******************************





In [127]:
df.groupby(["make", "model", "fuel", "age"])["emission_class"].value_counts().head(20)

make  model  fuel     age  emission_class
Audi  A1     Benzine  0.0  Euro 6            266
                           Euro 6d-TEMP      264
                           Euro 6d             5
                      1.0  Euro 6            323
                           Euro 6d-TEMP       11
                           Euro 5              3
                      2.0  Euro 6            213
                           Euro 5              1
                      3.0  Euro 6            257
                           Euro 6d-TEMP        3
                           Euro 4              1
             Diesel   0.0  Euro 6              3
                      1.0  Euro 6            295
                           Euro 5              1
                      2.0  Euro 6            133
                      3.0  Euro 6            269
                           Euro 5              4
                           Euro 6c             2
      A2     Diesel   1.0  Euro 6              1
      A3     Benzine  0.0  

In [128]:
df.groupby(["make", "model", "fuel", "age"])["emission_class"].value_counts().tail(20)

make     model   fuel     age  emission_class
Renault  Espace  Benzine  1.0  Euro 6d-TEMP        3
                               Euro 4              1
                               Euro 6c             1
                          2.0  Euro 6             38
                               Euro 5              1
                          3.0  Euro 6             29
                 Diesel   0.0  Euro 6             49
                               Euro 6d-TEMP       39
                               Euro 6c             2
                               Euro 6d             2
                          1.0  Euro 6             97
                               Euro 6c             3
                               Euro 6d             2
                          2.0  Euro 6             74
                               Euro 6c             1
                          3.0  Euro 6            197
                               Euro 6d-TEMP        3
                               Euro 5              1


In [129]:
df["emission_class"] = df.groupby(["make", "model", "fuel", "age"])["emission_class"].fillna(method="bfill").fillna(method="ffill")

## comfort&convenience

In [130]:
check_df("comfort&convenience")

column name :  comfort&convenience
--------------------------------
Per_of_Nulls   :  % 5.78
Num_of_Nulls   :  920
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                          

In [131]:
df["comfort&convenience"] = df.groupby(["make", "model", "age", "body_type"])["comfort&convenience"].fillna(method="pad").fillna(method="backfill")

In [132]:
df["comfort&convenience"].isnull().sum()

0

## entertainment&media

In [133]:
check_df("entertainment&media")

column name :  entertainment&media
--------------------------------
Per_of_Nulls   :  % 8.63
Num_of_Nulls   :  1374
NaN                                                                                                 1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                          1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                       982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                             783
On-board computer,Radio                                                                              487
                                                                                                    ... 
Bluetooth,CD player,Digital radio,Hands-free equipment,Radio                                           1
CD player,Hands-free equipment,MP3,Radio,USB                                                           1
MP3,Radio,Sound system,USB                  

In [134]:
 df["entertainment&media"] = df.groupby(["make", "model", "age", "body_type"])["entertainment&media"].fillna(method="pad").fillna(method="backfill")

In [135]:
df["entertainment&media"].isnull().sum()

0

## extras

In [136]:
# df[ df.extras.str.contains("Alloy wheels") if ~df.extras.isnull() else False ]

In [137]:
check_df("extras")

column name :  extras
--------------------------------
Per_of_Nulls   :  % 18.61
Num_of_Nulls   :  2962
Alloy wheels                                                                                               3245
NaN                                                                                                        2962
Alloy wheels,Touch screen                                                                                   697
Alloy wheels,Voice Control                                                                                  577
Alloy wheels,Touch screen,Voice Control                                                                     541
                                                                                                           ... 
Alloy wheels,Catalytic Converter,Shift paddles,Sport package,Sport seats,Sport suspension,Voice Control       1
Alloy wheels,Catalytic Converter,Roof rack,Sport package,Sport seats,Trailer hitch                            1


In [138]:
df.groupby(["make", "model", "age", "body_type"])["extras"].value_counts(dropna=False)

make     model   age  body_type  extras                                                            
Audi     A1      0.0  Compact    Alloy wheels                                                          65
                                 NaN                                                                   57
                                 Alloy wheels,Catalytic Converter,Voice Control                        54
                                 Alloy wheels,Voice Control                                            51
                                 Alloy wheels,Catalytic Converter                                      15
                                                                                                       ..
Renault  Espace  3.0  Van        Alloy wheels,Sport seats,Trailer hitch,Voice Control                   1
                                 Alloy wheels,Sport suspension,Touch screen                             1
                                 Alloy wheels,Sport 

In [139]:
 df["extras"] = df.groupby(["make", "model", "age", "body_type"])["extras"].fillna(method="pad").fillna(method="backfill")

In [140]:
df.extras.isnull().sum()

0

## safety&security

In [141]:
check_df("safety&security")

column name :  safety&security
--------------------------------
Per_of_Nulls   :  % 6.17
Num_of_Nulls   :  982
NaN                                                                                                                                                                                                                                                                                                                                                               982
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                      538
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressur

In [142]:
 df["safety&security"] = df.groupby(["make", "model", "age", "body_type"])["safety&security"].fillna(method="pad").fillna(method="backfill")

In [143]:
df["safety&security"].isnull().sum()

0

## gears         

In [144]:
check_df("gears")

column name :  gears
--------------------------------
Per_of_Nulls   :  % 29.6
Num_of_Nulls   :  4712
6.0     5822
NaN     4712
5.0     3239
7.0     1908
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: gears, dtype: int64
******************************





In [145]:
df.groupby(["make", "model", "body_type"])["gears"].value_counts()

make     model   body_type  gears
Audi     A1      Compact    5.0      280
                            7.0      202
                            6.0       83
                            8.0        1
                 Coupe      5.0        1
                                    ... 
Renault  Espace  Van        6.0      359
                            7.0      105
                            9.0        6
                            3.0        2
                            4.0        2
Name: gears, Length: 102, dtype: int64

In [146]:
df.groupby(["make", "model", "body_type"])["gears"].fillna(method="ffill").fillna(method = "bfill").value_counts()

6.0     8370
5.0     4398
7.0     2843
8.0      288
9.0       11
3.0        3
1.0        2
4.0        2
2.0        1
50.0       1
Name: gears, dtype: int64

In [147]:
df.groupby(["make", "model", "body_type"])["gears"].apply(lambda x : x.fillna(x.mode()[0] if any(x.mode()) else df.gears.mode()[0])).value_counts()

6.0     8411
5.0     5359
7.0     1911
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: gears, dtype: int64

In [148]:
df.gears = df.groupby(["make", "model", "body_type"])["gears"].fillna(method="ffill").fillna(method = "bfill")

In [149]:
df.gears.isnull().sum()

0

## country_version         

In [150]:
check_df("country_version")

column name :  country_version
--------------------------------
Per_of_Nulls   :  % 52.35
Num_of_Nulls   :  8333
NaN               8333
Germany           4502
Italy             1038
European Union     507
Netherlands        464
Spain              325
Belgium            314
Austria            208
Czech Republic      52
Poland              49
France              38
Denmark             33
Hungary             28
Japan                8
Slovakia             4
Croatia              4
Sweden               3
Romania              2
Bulgaria             2
Luxembourg           1
Switzerland          1
Slovenia             1
Egypt                1
Serbia               1
Name: country_version, dtype: int64
******************************





In [151]:
df.groupby(["make", "model", "body_type", "vat"])["country_version"].value_counts(dropna=False).head(60)

make  model  body_type      vat               country_version
Audi  A1     Compact        Price negotiable  Germany             11
                                              NaN                  8
                                              Netherlands          3
                            VAT deductible    Germany            437
                                              NaN                418
                                              Netherlands         65
                                              European Union      58
                                              Denmark             15
                                              Italy                8
                                              Austria              5
                                              Belgium              5
                                              France               2
                                              Poland               2
                                         

In [152]:
df.country_version = df.country_version.fillna("Unknown")

In [153]:
df.country_version.isnull().sum()

0

In [154]:
df.country_version.value_counts()

Unknown           8333
Germany           4502
Italy             1038
European Union     507
Netherlands        464
Spain              325
Belgium            314
Austria            208
Czech Republic      52
Poland              49
France              38
Denmark             33
Hungary             28
Japan                8
Slovakia             4
Croatia              4
Sweden               3
Romania              2
Bulgaria             2
Luxembourg           1
Switzerland          1
Slovenia             1
Egypt                1
Serbia               1
Name: country_version, dtype: int64

## consumption_comb, consumption_city, consumption_country

In [155]:
df[['consumption_comb', 'consumption_city','consumption_country']].isnull().sum()

consumption_comb       3790
consumption_city       4039
consumption_country    3365
dtype: int64

In [156]:
df.consumption_comb.mean()

4.860722235963347

In [157]:
df.consumption_city.mean()

5.973047138047285

In [158]:
df.consumption_country.mean()

4.214107057511512

In [159]:
# ratio of relation between consumption_city and consumption_country is 0.38 and 0.62 

In [160]:
df.groupby(['make','model','cylinders','weight_kg', 'body_type'])['consumption_city'].value_counts(dropna=False).head(60)

make  model  cylinders  weight_kg  body_type      consumption_city
Audi  A1     3.0        102.0      Compact        5.6                  1
                        1010.0     Compact        NaN                  2
                                   Coupe          NaN                  2
                        1035.0     Compact        NaN                  6
                                   Sedans         NaN                  5
                        1040.0     Compact        NaN                  1
                                                  5.4                  1
                        1060.0     Sedans         NaN                  2
                        1065.0     Compact        5.4                 25
                                                  5.5                  7
                                                  NaN                  3
                                                  5.6                  1
                                   Sedans         5.4    

In [161]:
df.groupby(["make", "model", "cylinders", "weight_kg", "body_type", "fuel"])["consumption_city"].value_counts()

make     model   cylinders  weight_kg  body_type    fuel     consumption_city
Audi     A1      3.0        102.0      Compact      Benzine  5.6                  1
                            1040.0     Compact      Benzine  5.4                  1
                            1065.0     Compact      Benzine  5.4                 25
                                                             5.5                  7
                                                             5.6                  1
                                                                                 ..
Renault  Espace  4.0        2037.0     Van          Diesel   6.2                  1
                            2044.0     Sedans       Diesel   6.3                  1
                            2353.0     Transporter  Diesel   5.1                 14
                            2410.0     Transporter  Benzine  7.8                  3
                            2471.0     Van          Diesel   5.1                  

In [162]:
df[['consumption_comb', 'consumption_city','consumption_country']]

,consumption_comb,consumption_city,consumption_country
0,3.8,4.3,3.5
1,5.6,7.1,4.7
2,3.8,4.4,3.4
3,3.8,4.3,3.5
4,4.1,4.6,3.8
...,...,...,...
15914,5.3,6.2,4.7
15915,7.4,9.2,6.3
15916,5.3,6.2,4.7
15917,5.3,6.2,4.7


In [163]:
df["consumption_comb"] = df["consumption_comb"].fillna(abs(df["consumption_country"]*0.62 + df["consumption_city"]*0.38))

In [164]:
df.consumption_comb.value_counts()

5.400    769
3.900    733
5.100    654
4.400    623
5.600    618
        ... 
7.500      1
3.780      1
5.074      1
4.282      1
5.098      1
Name: consumption_comb, Length: 115, dtype: int64

In [165]:
df.consumption_comb.isnull().sum()

2490

In [166]:
df["consumption_country"] = df["consumption_country"].fillna(abs((df["consumption_comb"] - df["consumption_city"]*0.38)/0.62))

In [167]:
df.consumption_country.isnull().sum()

2876

In [168]:
df["consumption_city"] = df["consumption_city"].fillna(abs(df["consumption_comb"] - df["consumption_country"]*0.62)/0.38)

In [169]:
df.consumption_city.isnull().sum()

2942

In [170]:
df.consumption_city = df.groupby(["make", "model", "body_type" ,"weight_kg", "cylinders", "fuel"])["consumption_city"].fillna(method="ffill").fillna(method="bfill")

In [171]:
df.consumption_city.isnull().sum()

0

In [172]:
df.consumption_country = df.groupby(["make", "model", "body_type" ,"weight_kg", "cylinders", "fuel"])["consumption_country"].fillna(method="ffill").fillna(method="bfill")

In [173]:
df.consumption_country.isnull().sum()

0

In [174]:
df["consumption_comb"] = df["consumption_comb"].fillna(abs(df["consumption_country"]*0.62 + df["consumption_city"]*0.38))

In [175]:
df.consumption_comb.isnull().sum()

0

In [176]:
df.isnull().sum()

body_type                 0
price_€                   0
vat                       0
km                        0
hp_kW                     0
type                      0
previous_owners           0
warranty                  0
make                      0
model                     0
body_color                0
paint_type                0
upholstery                0
nr_of_doors               0
nr_of_seats               0
gearing_type              0
engine_displacement_cc    0
cylinders                 0
weight_kg                 0
drive_chain               0
fuel                      0
co2e_g                    0
emission_class            0
comfort&convenience       0
entertainment&media       0
extras                    0
safety&security           0
gears                     0
country_version           0
consumption_comb          0
consumption_city          0
consumption_country       0
age                       0
dtype: int64

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   body_type               15919 non-null  object 
 1   price_€                 15919 non-null  float64
 2   vat                     15919 non-null  object 
 3   km                      15919 non-null  float64
 4   hp_kW                   15919 non-null  float64
 5   type                    15919 non-null  object 
 6   previous_owners         15919 non-null  float64
 7   warranty                15919 non-null  float64
 8   make                    15919 non-null  object 
 9   model                   15919 non-null  object 
 10  body_color              15919 non-null  object 
 11  paint_type              15919 non-null  object 
 12  upholstery              15919 non-null  object 
 13  nr_of_doors             15919 non-null  float64
 14  nr_of_seats             15919 non-null

In [178]:
df["consumption_city"].sort_values(ascending=True).head(60)

11553    0.321053
9756     0.710526
13008    1.589474
10755    1.663158
10758    1.663158
10759    1.663158
10764    1.673684
10870    1.673684
10866    1.673684
10860    1.673684
4936     1.805263
3483     1.805263
9268     2.178947
4026     2.294737
4741     2.294737
13680    2.447368
13678    2.447368
13679    2.447368
11503    2.784211
11504    2.784211
11596    2.784211
12068    2.784211
11088    2.784211
11065    2.784211
10811    2.784211
2245     3.057895
13356    3.300000
13355    3.300000
13354    3.300000
13353    3.300000
13352    3.300000
13349    3.300000
13347    3.300000
13357    3.300000
13312    3.300000
13309    3.300000
13307    3.300000
13579    3.300000
6153     3.300000
13348    3.300000
13358    3.300000
13360    3.300000
13518    3.300000
13523    3.300000
6160     3.300000
13715    3.300000
13501    3.300000
6159     3.300000
13716    3.300000
13721    3.300000
13442    3.300000
13668    3.300000
13369    3.300000
13366    3.300000
13861    3.300000
13365    3

In [179]:
df.iloc[[10764, 10755]]

,body_type,price_€,vat,km,hp_kW,type,previous_owners,warranty,make,model,body_color,paint_type,upholstery,nr_of_doors,nr_of_seats,gearing_type,engine_displacement_cc,cylinders,weight_kg,drive_chain,fuel,co2e_g,emission_class,comfort&convenience,entertainment&media,extras,safety&security,gears,country_version,consumption_comb,consumption_city,consumption_country,age
10764,Sedans,12100.0,VAT deductible,103000.0,92.0,Used,1.0,1.0,Opel,Insignia,Black,Metallic,Cloth,5.0,5.0,Automatic,1956.0,4.0,1503.0,front,Diesel,147.0,Euro 6,Power windows,"On-board computer,Radio",Alloy wheels,"ABS,Central door lock,Driver-side airbag,Immob...",6.0,Unknown,4.2,1.673684,7.8,2.0
10755,Sedans,11700.0,VAT deductible,107000.0,92.0,Used,1.0,1.0,Opel,Insignia,White,Metallic,Cloth,5.0,5.0,Automatic,1956.0,4.0,1503.0,front,Diesel,164.0,Euro 6,Power windows,"On-board computer,Radio",Alloy wheels,"ABS,Central door lock,Driver-side airbag,Immob...",6.0,Unknown,4.7,1.663158,8.6,2.0


In [180]:
pd.set_option('display.max_rows', 130)

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   body_type               15919 non-null  object 
 1   price_€                 15919 non-null  float64
 2   vat                     15919 non-null  object 
 3   km                      15919 non-null  float64
 4   hp_kW                   15919 non-null  float64
 5   type                    15919 non-null  object 
 6   previous_owners         15919 non-null  float64
 7   warranty                15919 non-null  float64
 8   make                    15919 non-null  object 
 9   model                   15919 non-null  object 
 10  body_color              15919 non-null  object 
 11  paint_type              15919 non-null  object 
 12  upholstery              15919 non-null  object 
 13  nr_of_doors             15919 non-null  float64
 14  nr_of_seats             15919 non-null

## Export dataframe to csv file (without dummy)

In [183]:
df.to_csv("filled_autoscout.csv", index=False)